In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
import nltk
from nltk.corpus import gutenberg

# Get the raw text of "Emma" by Jane Austen
text = gutenberg.raw("austen-emma.txt")

# Print the first 500 characters of the text
print(text[:500])

[Emma by Jane Austen 1816]

VOLUME I

CHAPTER I


Emma Woodhouse, handsome, clever, and rich, with a comfortable home
and happy disposition, seemed to unite some of the best blessings
of existence; and had lived nearly twenty-one years in the world
with very little to distress or vex her.

She was the youngest of the two daughters of a most affectionate,
indulgent father; and had, in consequence of her sister's marriage,
been mistress of his house from a very early period.  Her mother
had died t


In [3]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '6', '7', '8', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [4]:
vocab_size

77

In [5]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [6]:
x = 'Hello World!'

In [7]:
encoded = encode(x)

In [8]:
encoded

[29, 55, 62, 62, 65, 1, 44, 65, 68, 62, 54, 2]

In [9]:
decode(encoded)

'Hello World!'

In [10]:
data = torch.tensor(encode(text), dtype=torch.long)

In [11]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[55, 68, 75,  1, 73, 65, 68, 54],
        [70, 55, 68, 55, 69, 70,  1, 75],
        [ 3,  0,  0,  3, 24, 58, 68, 59],
        [ 1, 73, 51, 68, 63,  1, 51, 70]], device='cuda:0')
targets:
tensor([[68, 75,  1, 73, 65, 68, 54, 10],
        [55, 68, 55, 69, 70,  1, 75, 65],
        [ 0,  0,  3, 24, 58, 68, 59, 69],
        [73, 51, 68, 63,  1, 51, 70, 70]], device='cuda:0')


In [12]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


4D[T.doygaMWt!mY.-!ePn-;x]k[D3m!MHFLqLaL-fuGC4b"LV0` Vx28Fla?)2y UbO4wG6tTvxFQE7]E.wOTOz;.ge;hSNrGrT&GT.,8CM"4Io&i&tqKMl6RN[ PSDzO 1V`tLQ!uqK`8egwea?C?Uy3vQ2
B6U'tLRpvuoNKV)LeiX:qvrrczSM_-`g,RE`,Rioa2KtcH'iXHUHdoUHmKDQx]cD2s"o]n)-m:wq[(S&1C-d?H0srS-0b uXes i2Xc:Jzh"iJy?gLgx'8,0JBxtUyS;dAh2QY[zQzH,&i67nfWS_7srqaP0btU'dK0s?l6,elKP.DNW d-"kgx'0mCdhi-WoOfXRw!Va.Q?FaQaUm0Dv0IPHEnjY.'GzksHgK(n?P.llKGzXR"&3fIIQTCISp2lKtXl&Isbk4!bJ&L
O-k;ErNO0HD)k-WsM.-mOo dE`,m';wyp,Vlnj&6,H)nra'Mk!zXA"oIxFqSDbg,HFqg?.
